# Load files to pandas data frame

In [2]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_b3d150584aa142a1975f62c1c8b1699a(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_e28904fb9993787d907536191c825c593c36c601','domain': {'id': '0592036926cf47bf87e889ec75d0d38b'},
            'password': 'W-ucS.1Ge_fr.0b('}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_b3d150584aa142a1975f62c1c8b1699a('201710kbankibm', 'cc_info.csv'))
df_data_1.head()


,card_no,card_type,opn_dt,exp_dt,cr_lmt_amt,prev_cr_lmt_amt,main_zip_cd,cr_line_amt,incm_amt,brth_estb_yr
0,4410123456000001,visa,1997-10-14,1017,50000,0,73120.0,50000,21000,1953.0
1,4410123456000002,master,2010-06-29,620,146000,0,43000.0,146000,72000,1965.0
2,4410123456000003,visa,2014-06-18,619,22000,0,23000.0,22000,20000,1978.0
3,4410123456000004,visa,2014-11-03,1119,1000000,0,10200.0,2141000,1800000,1955.0
4,4410123456000005,visa,2014-04-10,419,80000,0,57160.0,80000,32000,1969.0


# Load to spark rdd

In [5]:

import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '1e25eccc8cec40699e74914679c79593',
    'region': 'dallas',
    'user_id': 'f909479c2549457cb3fc79a7445beeff',
    'username': 'member_e28904fb9993787d907536191c825c593c36c601',
    'password': 'W-ucS.1Ge_fr.0b('
}

configuration_name = 'os_b3d150584aa142a1975f62c1c8b1699a_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_2 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(bmos.url('201710kbankibm', 'cc_info.csv'))
df_data_2.take(5)


[Row(card_no=u'4410123456000001', card_type=u'visa', opn_dt=u'1997-10-14', exp_dt=u'1017', cr_lmt_amt=u'50000', prev_cr_lmt_amt=u'0', main_zip_cd=u'73120.0', cr_line_amt=u'50000', incm_amt=u'21000', brth_estb_yr=u'1953'),
 Row(card_no=u'4410123456000002', card_type=u'master', opn_dt=u'2010-06-29', exp_dt=u'620', cr_lmt_amt=u'146000', prev_cr_lmt_amt=u'0', main_zip_cd=u'43000.0', cr_line_amt=u'146000', incm_amt=u'72000', brth_estb_yr=u'1965'),
 Row(card_no=u'4410123456000003', card_type=u'visa', opn_dt=u'2014-06-18', exp_dt=u'619', cr_lmt_amt=u'22000', prev_cr_lmt_amt=u'0', main_zip_cd=u'23000.0', cr_line_amt=u'22000', incm_amt=u'20000', brth_estb_yr=u'1978'),
 Row(card_no=u'4410123456000004', card_type=u'visa', opn_dt=u'2014-11-03', exp_dt=u'1119', cr_lmt_amt=u'1000000', prev_cr_lmt_amt=u'0', main_zip_cd=u'10200.0', cr_line_amt=u'2141000', incm_amt=u'1800000', brth_estb_yr=u'1955'),
 Row(card_no=u'4410123456000005', card_type=u'visa', opn_dt=u'2014-04-10', exp_dt=u'419', cr_lmt_amt=u'8

In [15]:

df_data_3 = pd.read_csv(get_object_storage_file_with_credentials_b3d150584aa142a1975f62c1c8b1699a('201710kbankibm', 'answer.csv'))
df_data_3.shape[0]


99594

In [22]:
cc_info = pd.concat((df_data_1, df_data_3), axis=1)

In [23]:
cc_info.head()

,card_no,card_type,opn_dt,exp_dt,cr_lmt_amt,prev_cr_lmt_amt,main_zip_cd,cr_line_amt,incm_amt,brth_estb_yr,Automobiles and Vehicles-freq,Clothing Stores-freq,Service Providers-freq,Transportation-freq,Utilities-freq,Automobiles and Vehicles-sum,Clothing Stores-sum,Service Providers-sum,Transportation-sum,Utilities-sum
0,4410123456000001,visa,1997-10-14,1017,50000,0,73120.0,50000,21000,1953.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,4410123456000002,master,2010-06-29,620,146000,0,43000.0,146000,72000,1965.0,0,0,1,0,0,0.0,0.0,1580.0,0.0,0.0
2,4410123456000003,visa,2014-06-18,619,22000,0,23000.0,22000,20000,1978.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,4410123456000004,visa,2014-11-03,1119,1000000,0,10200.0,2141000,1800000,1955.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,4410123456000005,visa,2014-04-10,419,80000,0,57160.0,80000,32000,1969.0,0,0,0,1,0,0.0,0.0,0.0,780.0,0.0


In [12]:
df_data_1.groupby("card_no").agg({"cr_lmt_amt": "sum", "cr_line_amt":"mean", "incm_amt": "mean"})

,cr_lmt_amt,incm_amt,cr_line_amt
card_no,,,
4410123456000001,50000,21000,50000
4410123456000002,146000,72000,146000
4410123456000003,22000,20000,22000
4410123456000004,1000000,1800000,2141000
4410123456000005,80000,32000,80000
4410123456000006,100000,66000,100000
4410123456000007,65000,26000,65000
4410123456000008,48000,20000,48000
4410123456000009,327000,109000,327000
